In [ ]:
#This notebook reproduces the maps shown in Figures 2, 3 and 5 of the paper.
'''
Demonstrated features:
1. Computing sampling fairness and geochemical consensus scores using
   the algorithm.run() API.
2. Using the method `draw_grade_blocks` from the "custom_graphs.py" module
   to draw block boundaries and show the spatial confidence and geochemical
   consensus values within the grade-blocks in a colour map.
''';

__copyright__ = "Copyright (c) 2022, Raymond Leung"
__license__   = "GPLv3"

In [ ]:
import numpy as np
import pandas as pd
import shapely.wkt
import sys

from shapely.geometry import Polygon
from algorithm_implementation import GradeBlockReliabilityAssessment

In [ ]:
#read test data
#- blasthole assays
#- grade-block boundaries
df_bh = pd.read_csv('data/anonymised_blastholes.csv')
df_gb = pd.read_csv('data/anonymised_gradeblocks.csv')
uniq_gradeblock_ids = df_gb['gradeblock_id'].values

#define lambda functions to access information from dataframes
retrieve = lambda df, cols: [df[df['gradeblock_id']==g][cols].values
                             for g in uniq_gradeblock_ids]
holes_properties = lambda cols: retrieve(df_bh, cols)
block_properties = lambda cols: retrieve(df_gb, cols)
block_names = dict(zip(uniq_gradeblock_ids, block_properties('name')))

block_boundaries = dict(zip(df_gb['gradeblock_id'].values,
                        [shapely.wkt.loads(wkt) for wkt in df_gb['boundary'].values]))
blasthole_xy = dict(zip(uniq_gradeblock_ids, holes_properties(['x','y'])))
blasthole_assays = dict(zip(uniq_gradeblock_ids, holes_properties(['c1','c2','c3'])))

In [ ]:
#configuration
config = {'modelled_elements': ['Fe', 'SiO2', 'Al2O3'],
          'chemical_weights': [0.5, 0.325, 0.175]}
results = {
       'grade_block_id': None,
       'score_spatial_confidence': np.nan,
       'score_chemical_consensus': np.nan,
       'score_overall_reliability': np.nan,
       'factor_gradeblock_coverage': np.nan,
       'factor_hole_entropies': np.nan,
       'factor_sampling_density': np.nan,
       'stats_hole_sparsity': np.nan,
       'stats_rd_gmean': np.nan,
       'stats_rd_score': np.nan,
       'stats_eta_outliers': np.nan,
       'stats_area': np.nan,
       'stats_hole_count': np.nan,
       'stats_effective_samples': np.nan
       }

In [ ]:
#instantiate object
algorithm = GradeBlockReliabilityAssessment(config)

gb_ids = []
gb_names = []
gb_vertices = []
gb_spatial_confidence = []
gb_chemical_consensus = []

#analyse individual grade-blocks
for i, gb in enumerate(uniq_gradeblock_ids):
    results['grade_block_id'] = gb
    algorithm.run(block_boundaries[gb], blasthole_xy[gb], blasthole_assays[gb], results)
    gb_ids.append(gb)
    gb_names.append(block_names[gb])
    gb_vertices.append(np.array(Polygon(block_boundaries[gb]).exterior.coords.xy).squeeze().T)
    gb_spatial_confidence.append(results['score_spatial_confidence'])
    gb_chemical_consensus.append(results['score_chemical_consensus'])
    #show progress bar
    pct = (i * 100.) / (len(uniq_gradeblock_ids)-1)
    steps = int(np.floor(pct / 5))
    sys.stdout.write('\r')
    sys.stdout.write("[%-20s] %d%% completed" % ('*' * steps, pct))
    sys.stdout.flush()

In [ ]:
valid = np.where(~np.isnan(gb_spatial_confidence) & ~np.isnan(gb_chemical_consensus))[0]
gb_ids = np.array([gb_ids[r] for r in valid])
gb_polys = [gb_vertices[r] for r in valid]
gb_abbrevs = [gb_names[r][0] for r in valid]
gb_spatial_confidence = np.array([gb_spatial_confidence[r] for r in valid])
gb_chemical_consensus = np.array([gb_chemical_consensus[r] for r in valid])

In [ ]:
from custom_graphs import draw_grade_blocks

#draw_grade_blocks(gb_polys, gb_abbrevs, gb_fill=None, show=True, categorise=True, fontsz=5)

draw_grade_blocks(gb_polys, gb_abbrevs, gb_spatial_confidence, show=True,
                  alpha=1, cbar_title='spatial confidence', gamma=3.0, fontsz=5)

draw_grade_blocks(gb_polys, gb_abbrevs, gb_chemical_consensus, show=True,
                  alpha=1, cbar_title='geochemical consensus', gamma=3.0, fontsz=5)

draw_grade_blocks(gb_polys, gb_abbrevs, gb_spatial_confidence * gb_chemical_consensus, show=True,
                  alpha=1, cbar_title='overall grade-block reliability', gamma=3.0, fontsz=5)